### META-LEARNING SIMULATION

In [ ]:
import numpy as np
import os 
import matplotlib.pyplot as plt
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim

device='cuda'

from Util_MetLearning import *


### LOADING THE DATA

The cell below loads the devices responses. If Single_R=False, the data will be loaded from all the devices in the physical neural network, if Single_R=True, the code will load the data from a single device.

The PNN is driven by an external sinuoisodal signal $$s(t)=\sin(\omega t) $$, and it is tasked to reproduce different outputs $y_i(t)=a_i\sin(\omega_i t+\theta_i)$ in a few-shot learning fashion and as described in the paper.




In [ ]:
datafolder='Sine_hierachies'
X=[]
T_data=250
X=np.zeros([T_data,1])
S=np.zeros([T_data,1])

Single_R=False

if Single_R==True:
        
        ## Loading signle device response
        X1=np.load(os.path.join(datafolder,'7.ASVI04_From_HDS_Mem_peak_262.npy'),allow_pickle=True) 
        
else:
    
    
    ## Loading physical neural network response
    for file in os.listdir(datafolder):

        if '.npy' in file: 

            X1=np.load(os.path.join(datafolder,file),allow_pickle=True)
            if np.shape(X1)[0]>=T_data and file !='all_targets.npy' and file!='target_names.npy' and file!='3.Pinwheel_ASVI_y.npy':

                X=np.concatenate([X,X1[0:T_data,1:]],1)  

S1=np.load(os.path.join(datafolder,'3.Pinwheel_ASVI_y.npy'),allow_pickle=True)
X=np.concatenate([X,S1[0:T_data,2:]],1)

S=S1[0:T_data,0]
S=(S-np.min(S))/(np.max(S)-np.min(S))


## Plot external Signal and response example
fig, ax = plt.subplots(1, 2,figsize=(10, 4))

ax[0].plot(S[0:100],'.')
dts=torch.arange(0,250)*2*np.pi/30
ax[0].plot((torch.sin(dts[0:100])+1)/2)
ax[0].set_xlabel('steps')
ax[0].set_ylabel('External signal')
ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)


X=torch.tensor(X[:,1:]).float().to('cuda')
X=X-torch.min(X,0)[0].unsqueeze(0)
X_M=torch.max(torch.abs(X),0)[0]
X=X/X_M.unsqueeze(0)

Dim_plot=50
rand_dims=np.random.randint(0,X.size()[0],Dim_plot)

for n in range(Dim_plot):
    
    
    ax[1].plot(X[0:100,rand_dims[n]].to('cpu'))

ax[1].set_xlabel('steps')
ax[1].set_ylabel('Devices Responses')    
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)



In [ ]:
Sines=SineWaves(dts,X)
space=3
N_task=10
rand_sample=True
Rand=True
plt.rcParams["figure.autolayout"] = True


import matplotlib as mpl
colors = mpl.colormaps['magma'].resampled(Sines.N_F)


## Randomly sample a sine wave in the range of values considered 
if rand_sample==True:
    
    a=np.zeros([Sines.N_F])
    theta=np.zeros([Sines.N_F])
    
    for n in range(Sines.N_F):
        
        a[n]=np.random.uniform(Sines.As[n][0],Sines.As[n][1])
        theta[n]=np.random.uniform(Sines.phases[n][0],Sines.phases[n][1])

batch_size1=10
batch_size2=10

fig, ax = plt.subplots(4, 1,figsize=(10, 12))

S_T, T_T, x_data, y_data, n_t=Sines.Sample_1(a,theta,batch_size1,space)

## Plot thdata 

for  n in range(Sines.N_F-1):
    if n==0:
        ax[0].plot(dts[n_t].to('cpu'),T_T[:,n].to('cpu'),'.',markersize=10,color='black')
    
    ax[0].plot(dts.to('cpu'),y_data[:,n].to('cpu'),color=colors(n))

ax[0].set_xlabel('steps')
ax[0].set_ylabel('Targets for different frequencies')
ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)
ax[0].set_xlim(0, 30)
ax[0].set_title('Example 1')
    
T_T, S_T, N_t, T_q, S_q, N_q, As, Ths=Sines.Sample_Tr(N_task,batch_size1,batch_size2,space,Rand)

n_task=np.random.randint(0,N_task)
for n in range(2,Sines.N_F):
    if n==2:
        ax[1].plot(dts[N_t[n_task]].to('cpu'), T_T[n_task][:,n].to('cpu'), '.',markersize=8,color='black')
        ax[1].plot(dts[N_q[n_task]].to('cpu'), T_q[n_task][:,n].to('cpu'), '.',markersize=8,color='red')
    Y=As[n_task][n]*(torch.sin((n+1)*dts+Ths[n_task][n])+1)/2
    ax[1].plot(dts.to('cpu'), Y.to('cpu'),color=colors(n))


    
ax[1].set_xlabel('steps')
ax[1].set_ylabel('Targets for different frequencies')
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
ax[1].set_xlim(0, 30)
ax[1].set_title('Example 2')

#print(As[n_task], Ths[n_task])

Ss, Targets, N_T, x_datas, y_datas, As, Ths=Sines.sample_all(N_task,batch_size1,space,Rand)

Y1=torch.zeros([dts.size()[0]])
for n in range(0,Sines.N_F):
    if n==0:
        ax[2].plot(dts[N_T[n_task]].to('cpu'), Targets[n_task][:,n].to('cpu'), '.',markersize=8,color='black')
    Y=As[n_task][n]*(torch.sin((n+1)*dts+Ths[n_task][n])+1)/2
    Y1=Y1+Y
    ax[2].plot(dts.to('cpu'), Y.to('cpu'),color=colors(n))

ax[2].set_xlabel('steps')
ax[2].set_ylabel('Targets for different frequencies')
ax[2].spines['right'].set_visible(False)
ax[2].spines['top'].set_visible(False)    
ax[2].set_xlim(0, 30)
ax[2].set_title('Example 3')

#print(As[n_task], Ths[n_task])

ax[3].plot(dts.to('cpu'),Y1.to('cpu'))
ax[3].set_xlabel('steps')
ax[3].set_ylabel('Example of overall target')
ax[3].spines['right'].set_visible(False)
ax[3].spines['top'].set_visible(False)    
ax[3].set_xlim(0, 30)


In [ ]:
Ns=torch.tensor([X.size()[1],5,Sines.N_F])

K=15
Q=20
N_task=15
eta=1
eta_meta=0.000005
N_train=10000
t_steps=10
c=10
space=3

MLP=Meta_ReadOut(Ns,t_steps,eta)

MLP.Initialise_Hyperparameters(eta_meta,c,N_train,t_steps)

Sines=SineWaves(dts,X)

E_train=torch.zeros([N_train,N_task,t_steps])
L_train=torch.zeros([N_train,N_task,t_steps])
n_steps=1000

for n in range(N_train):
    

    T_T, S_T, N_t, T_q, S_q, N_q, As, Ths=Sines.Sample_Tr(N_task,K,Q,space,Rand=False)
    
    e_n, l=MLP.Meta(S_T, T_T, S_q, T_q, t_steps)
    
    E_train[n,:,:]=e_n[:,:].detach()
    L_train[n,:,:]=l[:,:].detach()    
    
    if n%500==0:
        
        Ss, Targets, N_T, x_datas, y_datas, As, Ths=Sines.sample_all(N_task*3,K,space,Rand=True)
        ES_tr, ES_all_tr, Ys_tr=MLP.Analysis(Ss, Targets, n_steps, x_datas, y_datas)
        
        print(n,'Performance:')
        
        ## Error on all the wave; it will give an idea of the generalisation ability over data not seen in
        ## the inner loop
        print('On all data:', torch.mean(ES_all_tr[0:900],0))
        
        ## Error on inner loop data; 
        ## The discrepancy between the two metrics gives an idea of overfitting tendency
        print('On data seen during training:', torch.mean(ES_tr[0:900],0))
        

In [ ]:
from scipy import io

n_points=15
space=2


rand_sample=True
if rand_sample==True:
    
    a=np.zeros([Sines.N_F])
    theta=np.zeros([Sines.N_F])
    
    for n in range(Sines.N_F):
        
        a[n]=np.random.uniform(Sines.As[n][0],Sines.As[n][1])
        theta[n]=np.random.uniform(Sines.phases[n][0],Sines.phases[n][1])
        
    
xs_t, s_t, x_data, y_data, n_t=Sines.Sample_1(a,theta,n_points,space,Rand=True)

n_steps=1000

E_n, E_all, Y_data=MLP.Fine_Tuning(xs_t, s_t, n_steps, x_data.to('cuda'), y_data.to('cuda'))


fig, ax = plt.subplots(6, 1,figsize=(12, 15))

print('Different colors correspond to the network output after siverse nuber of updates')
x_lim=10
X_lim=50
for n in range(Sines.N_F):
    
    ax[n].plot(dts,y_data[:,n].detach().to('cpu'),'black')
    ax[n].plot(dts,Y_data[2,:,n].detach().to('cpu'),'--')
    ax[n].plot(dts,Y_data[9,:,n].detach().to('cpu'),'--')
    ax[n].plot(dts,Y_data[999,:,n].detach().to('cpu'),'--')
    ax[n].plot(dts[n_t],s_t[:,n].to('cpu'),'.',color='red')
    ax[n].set_xlim([x_lim, X_lim])
    ax[n].set_xlabel('steps')
    ax[n].set_ylabel('Target for frequency omega '+str(n))
    ax[n].spines['right'].set_visible(False)
    ax[n].spines['top'].set_visible(False)

    
ax[n+1].plot(dts, torch.sum(y_data,1).to('cpu'),'black' ) 
ax[n+1].plot(dts, torch.sum(Y_data[2,:,:],1).to('cpu'),'--' ) 
ax[n+1].plot(dts, torch.sum(Y_data[9,:,:],1).to('cpu'),'--' ) 
ax[n+1].plot(dts, torch.sum(Y_data[999,:,:],1).to('cpu'),'--' ) 
ax[n+1].plot(dts[n_t],torch.sum(s_t,1).to('cpu'),'.',color='red')
ax[n+1].set_xlabel('steps')
ax[n+1].set_ylabel('Overall Target')
ax[n+1].spines['right'].set_visible(False)
ax[n+1].spines['top'].set_visible(False)
print(a,theta)




### SAVING RESULTS 

In [ ]:
title_start='Meta_Sines_Model'
title=title_start+'_Pred.npy'

Parameters=np.array( [np.array(Model.Ws.to('cpu')),np.array(Model.bs.to('cpu'))], dtype=object)

np.save(os.path.join(path, title), Pred, allow_pickle=True)

In [ ]:
from scipy import io

for j in range(1,100):

    n_points=j

    xs_t, s_t, x_data, y_data, n_t=Sines.Sample_1(a,theta,n_points,space,Rand=False)

    n_steps=1000

    E_n, E_all, Y_data=MLP.Fine_Tuning(xs_t, s_t, n_steps, x_data.to('cuda'), y_data.to('cuda'))
    Pred=np.array( [np.array(dts.to('cpu')),n_t,np.array(Y_data.to('cpu')),np.array(y_data.to('cpu')),\
                    np.array(E_n.to('cpu')),np.array(E_all.to('cpu')),a,theta], dtype=object)

    path="C:\\Users\lucam\Desktop\Meta\Data2"

    title_start='Meta_Sines_Ex6_'+str(n_points)
    title=title_start+'_space2.mat'
    io.savemat(os.path.join(path, title),{"array": Pred})



In [ ]:

path="C:\\Users\lucam\Desktop\Meta\Random_Examples"


Ex=np.arange(0,15)

for n in range(np.shape(Ex)[0]):
    
    #path="C:\\Users\lucam\Desktop\Clean\Meta_Results\Sines\Second_Data"
    
    path="C:\\Users\lucam\Desktop\Meta\Random_Examples"
    title_start='Meta_Sines_Ex'+str(Ex[n]+1)+'_10_Rand'
    title=title_start+'.mat'

    Pred=io.loadmat(os.path.join(path, title))
    a=Pred['array'][0][6][0]
    theta=Pred['array'][0][7][0]
    ns=np.arange(1,11)*3
    
    for j in range(10):

        n_points=ns[j]

        xs_t, s_t, x_data, y_data, n_t=Sines.Sample_1(a,theta,n_points,space,Rand=True)
        
        n_steps=1000

        E_n, E_all, Y_data=MLP.Fine_Tuning(xs_t, s_t, n_steps, x_data.to('cuda'), y_data.to('cuda'))
        Pred=np.array( [np.array(dts.to('cpu')),n_t,np.array(Y_data.to('cpu')),np.array(y_data.to('cpu')),\
                        np.array(E_n.to('cpu')),np.array(E_all.to('cpu')),a,theta], dtype=object)

        #path="C:\\Users\lucam\Desktop\Clean\Meta_Results\Sines\Meta0"
        path="C:\\Users\lucam\Desktop\Meta\Meta_single\Data"
        title_start='Meta_Single_Ex'+str(Ex[n])+'_'+str(n_points)
        title=title_start+'.mat'
        io.savemat(os.path.join(path, title),{"array": Pred})
    

In [ ]:
n_points=15
space=2

N_ex=500
for l in range(500,500+N_ex):
    
    for j in range(1,41):
        
        n_points=j
        rand_sample=True
        if rand_sample==True:

            a=np.zeros([Sines.N_F])
            theta=np.zeros([Sines.N_F])

            for n in range(Sines.N_F):

                a[n]=np.random.uniform(Sines.As[n][0],Sines.As[n][1])
                theta[n]=np.random.uniform(Sines.phases[n][0],Sines.phases[n][1])


        xs_t, s_t, x_data, y_data, n_t=Sines.Sample_1(a,theta,n_points,space,Rand=True)

        n_steps=1000

        E_n, E_all, Y_data=MLP.Fine_Tuning(xs_t, s_t, n_steps, x_data.to('cuda'), y_data.to('cuda'))

        #Pred=np.array( [np.array(dts.to('cpu')),n_t,np.array(Y_data.to('cpu')),np.array(y_data.to('cpu')),\
        #                    np.array(E_n.to('cpu')),np.array(E_all.to('cpu')),a,theta], dtype=object)
        
        
        Pred=np.array( [np.array(E_n.to('cpu')),np.array(E_all.to('cpu')),a,theta], dtype=object)

        
        path="C:\\Users\lucam\Desktop\Meta\Random_Errors"

        title_start='Meta_Sines_Ex'+str(l)+'_'+str(n_points)
        title=title_start+'_RandErrors.mat'
        io.savemat(os.path.join(path, title),{"array": Pred})

    
    
